In [1]:
import tensorflow as tf
import numpy as np
import pylab as pl

In [2]:
from sklearn.datasets import california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import scale

/home/wlwoon/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
california=california_housing.fetch_california_housing()

In [4]:
x=scale(california['data'])
y=california['target']
np.mean((y-y.mean())**2)

1.3315503000818076

In [5]:
# Splitting into train and test sets

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3)

# Random Forest regressor

* Benchmark system
* RF generally gives great performance for problems like this

In [6]:
xtrain.shape

(14448, 8)

In [7]:
rf=RandomForestRegressor(n_estimators=100).fit(xtrain,ytrain)

In [8]:
# MSE Error
np.mean((rf.predict(xtest)-ytest)**2)

0.26046762604209611

# "Deep" Neural Network

* Two hidden layers
* Investigate the effect of different params

In [9]:
nhidden1=150
nhidden2=100
n_iter=200
#activation=tf.nn.relu
activation=tf.nn.sigmoid

In [10]:
nx=tf.placeholder(tf.float32,(None,8),name='x')
ny=tf.placeholder(tf.float32,(None,1),name='y')
dropkeep=tf.placeholder(tf.float32)

w1=tf.Variable(tf.truncated_normal((8,nhidden1),stddev=0.01),name='w1')
b1=tf.Variable(tf.zeros(nhidden1),name='b1')
a1=tf.nn.dropout(activation(tf.matmul(nx,w1)+b1),dropkeep,name='a1')

w2=tf.Variable(tf.truncated_normal((nhidden1,nhidden2),stddev=.01),name='w2')
b2=tf.Variable(tf.zeros(nhidden2),name='b2')
a2=tf.nn.dropout(activation(tf.matmul(a1,w2)+b2),dropkeep,name='a2')

wout=tf.Variable(tf.truncated_normal((nhidden2,1),stddev=.01),name='w_out')
bout=tf.Variable(tf.zeros(1),name='b_out')

ypred=tf.matmul(a2,wout)+bout
loss=tf.losses.mean_squared_error(ny,ypred)
#regloss=tf.losses.mean_squared_error(ny,ypred)+.1*tf.nn.l2_loss(w1)+.1*tf.nn.l2_loss(w2)

opt=tf.train.AdamOptimizer().minimize(loss)

In [13]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter("/home/wlwoon/tmp/day14log", sess.graph)
    sess.run(tf.global_variables_initializer())
    for count in range(n_iter):
        _,tloss=sess.run([opt,loss],feed_dict={dropkeep:.5,nx:xtrain,ny:ytrain.reshape((-1,1))})
        if count % 100 ==0:
            test_loss=sess.run(loss,feed_dict={dropkeep:1.0,nx:xtest,ny:ytest.reshape((-1,1))})
            print("iter #"+str(count)+", training loss is "+str(tloss)+", test loss is "+str(test_loss))
    writer.flush()

iter #0, training loss is 5.54907, test loss is 5.29047
iter #100, training loss is 1.31329, test loss is 1.28582


In [14]:
count

199